In [19]:
# pip3 install nltk spacy gensim scikit-learn numpy
#Se importan las librerías necesarias
import os
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity

# Para correr el codigo se deben instalar estas librerias y descargar los discursos originales y colocarlos en la carpeta DiscursosOriginales en la misma carpeta que este archivo.

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fonchopinto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fonchopinto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
# Definir el directorio donde están los archivos .txt
directorio = 'DiscursosOriginales/'  # Reemplaza esto con tu directorio

# Lista para almacenar los contenidos tokenizados de los archivos
contenidos = []

# Leer los textos y combinarlos en un solo corpus para luego poder tokenizar y limpiar el contenido.
for filename in os.listdir(directorio):
  if filename.endswith('.txt'):
    with open(os.path.join(directorio, filename), 'r',
              encoding='utf-8') as file:
      contenido = file.read()
      print(f'Archivo {filename} leído')

      # Tokenizar y limpiar el contenido, donde:
      # Tokenizar: acción de dividir un texto en entidades más pequeñas ("tokens"), donde pueden ser palabras, subpalabras o caracteres. La elección de esta es fundamental ya que puede afectar el rendimiento del modelo.
      #lematizar: acción en donde se extrae la raíz o forma básica de la palabra.
      tokens = word_tokenize(
          contenido.lower())  # Convertir a minúsculas y tokenizar
      tokens_sin_stopwords = [
          word for word in tokens if word not in STOP_WORDS
      ]  #extrayendo stopwords de los tokens finales
      tokens_sin_stopwords = ' '.join(tokens_sin_stopwords)

      # Guardar el contenido tokenizado en la lista
      contenidos.append(tokens_sin_stopwords.replace(",", "").replace(".", "").replace("-", "").replace(":","").replace(";", ""))

Archivo 91265.txt leído
Archivo 83107.txt leído
Archivo 79915.txt leído
Archivo 153144.txt leído


Archivo 94447.txt leído
Archivo 84898.txt leído
Archivo 149691.txt leído
Archivo 89628.txt leído
Archivo 85012.txt leído
Archivo 85006.txt leído
Archivo 80142.txt leído
Archivo 96720.txt leído
Archivo 75444.txt leído
Archivo 86296.txt leído
Archivo 91891.txt leído
Archivo 164307.txt leído
Archivo 100454.txt leído
Archivo 80552.txt leído
Archivo 136643.txt leído
Archivo 84907.txt leído
Archivo 102280.txt leído
Archivo 101951.txt leído
Archivo 71541.txt leído
Archivo 90595.txt leído
Archivo 76776.txt leído
Archivo 96872.txt leído
Archivo 96866.txt leído
Archivo 85952.txt leído
Archivo 164266.txt leído
Archivo 94863.txt leído
Archivo 73020.txt leído
Archivo 101415.txt leído
Archivo 80396.txt leído
Archivo 153186.txt leído
Archivo 75121.txt leído
Archivo 100509.txt leído
Archivo 136905.txt leído
Archivo 86042.txt leído
Archivo 163938.txt leído
Archivo 81705.txt leído
Archivo 102450.txt leído
Archivo 74598.txt leído
Archivo 150300.txt leído
Archivo 98518.txt leído
Archivo 73977.txt leído
Ar

In [14]:
# Verificando tokenización, es decir, el proceso que elimina las "Stop words" que son aquellas palabras vacías o sin significado, como lo pueden ser los artículos, pronombres, preposiciones, etcétera.
# Para la lematización de los textos, al haber distintas librerías según el idioma, la de español (spaCy) con los tokens correspondientes creaba palabras inexistentes, lo que arruinaba el modelo (ejemplo: pandemia lo cambiaba a pandemiar). Y con esto, para la parte posterior de la consulta, para que resultara correcto había que poner una oración de extensión muy larga, incluso en muchos casos, casi el texto completo. Mejoró mucho la exactitud del modelo al eliminar la lematización por lo que decidimos no incluirla.
print(contenidos[7])

tardes  general mario alberto rozas córdova  nombre gobierno  seguro interpretando compatriotas  deseo exitosa gestión mando carabineros chile  quería compartir reflexiones   principal preocupación chilenos  probablemente seguir seguridad ciudadana  paz tranquilidad puedan desarrollar vidas libertad aspirar realización plena ciudadanos país   consecuencia  prioridad compromiso gobiernos  honor encabezar  esté alcance paz tranquilidad compatriotas  frente enemigo formidable  poderoso  cruel implacable demostrado respeta   cometer actos delincuencia  narcotráfico  crimen organizado terrorismo   consecuencia  cumplir importante misión  obligación gobierno  labor carabineros chile absolutamente fundamental  quiero expresar   profundo aprecio  reconocimiento gratitud realizan 60 mil carabineros ancho chile  visto ojos  desarrollan funciones entrega  abnegación  sacrificio realmente merece reconocimiento gratitud  visto elementos obligan profunda modernización carabineros chile  administrati

In [4]:
#Establecer el modelo de Word2Vec a partir de los 2 hiperparáetros principales el vector_size y el window. El primero representa el tamaño del vector en cuanto a su dimensión. En este caso entrenamos nuestro modelo con un vector_size de 300 ya que luego de iterar no se mejorara la calidad de los embeddings y la cosine similarity al incrementar el tamaño mas de 300 por lo que lo acotamos en 300 para no extender tantos los vectores sin motivo alguno y relentizar el codigo. Por otro lado el window es el tamaño de la ventana de contexto, es decir, cuantas palabras vecinas se toman en cuenta al predecir la siguiente palabra en una oracion. Esto evidentemente afecta la exactitud de nuestro modelo por lo que lo redujimos al mínimo de 2, ya que 1 se comportaba de forma extraña a veces acertando y otras no, mientras que en 2 no presentaba tantas fluctuaciones de exito.
model = Word2Vec(tokens, vector_size=300, window=2, min_count=1)

In [15]:
#En esta parte del código, se realiza una lista de los vectores y se les aplica el modelo entrenado de embeddings para generar un vector de 300 dimensiones por cada token y al final promediarlos y obtener un vector promedio por cada texto para la futura comparacion con la consulta.
vectores_por_texto = []
for texto_tokens in contenidos:
  # Se aplica el modelo para todos los tokens de un vector
  vectores_por_token = [
      model.wv[token] for token in texto_tokens if token in model.wv
  ]
  # A continuación se promedian los vectores para obtener un vector representativo del texto, en el cual, si existe, se asigna, si no, se agrega un vector de ceros.
  if vectores_por_token:
    # Promediar los vectores
    vector_promedio = sum(vectores_por_token) / len(vectores_por_token)
    # Se agregan los vectores al vector promedio
    vectores_por_texto.append(vector_promedio)
  else:
    # Se agrega un vector de ceros en caso de no existe un token.
    vectores_por_texto.append([0] * model.vector_size)

In [16]:
# Solicitamos una consulta al usuario y seguidamente calculamos la similitud, para calcular la similitud usamos la distancia coseno.
# Entre la consulta y cada discurso
consulta = str(input("Ingrese su consulta: "))
tokens_consulta = word_tokenize(consulta.lower())
# Se verifica palabra por palabra dentro de la consulta si esta en las palabras vacías.
tokens_sin_stopwords = [
    word for word in tokens_consulta if word not in STOP_WORDS
]
doc = ' '.join(tokens_sin_stopwords)
doc = doc.replace(",", "").replace(".", "").replace("-", "").replace(":","").replace(";", "")
embedding_consulta = [model.wv[token] for token in doc if token in model.wv] # Aquí se enera una lista de embeddings para cada token en "doc" si esque ese token está presente en el vocabulario del modelo.
vector_promedio = sum(embedding_consulta) / len(embedding_consulta)
embedding_consulta = vector_promedio.reshape(1, -1) #En esta parte se transforma el vector de representación de la consulta (vector_promedio) en una matriz de una fila y columnas según los elementos del vector original. Al final busca unidimensionalizar los datos.

In [17]:
#primero creamos una lista vacía llamada similitudes para almacenar los resultados de similitud.
#recorremos una lista de vectores, calculando la similitud coseno entre cada uno de ellos y un vector de consulta, y almacenamos el resultado de similitud junto con la información del contenido correspondiente en la lista.
similitudes = []
counter = 0
for vector in vectores_por_texto:
  # Se reajusta el vector de consulta al tamaño del vector de texto para poder realizar la cosine similarity correctamente.
  vector = vector.reshape(1, -1)
  similitud = cosine_similarity(embedding_consulta, vector)[0][0]
  similitudes.append((contenidos[counter], similitud))
  counter += 1

# Se ordena por similitud descendente y luego se muestran las 10 coincidencias más cercanas.
similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)

In [18]:
#Recorremos las primeras 10 tuplas (pares de valores) de la lista similitudes.
print(f'Consulta: {consulta}')
print('-*-' * 30)
for texto, similitud in similitudes[:
                                    10]:  # Muestra los 10 textos más relevantes según la similitud entregada.
  print(f'Similitud: {similitud}')
  print(texto)
  print('-' * 30)
# Al final el modelo entrenado presenta resultados positivos en la mayoria de casos, pero en otros negativos.
# Por lo tanto, es importante comprobar que el modelo sea capaz de clasificar y reconocer en forma precisa los textos a futuro, quizas cambiando de libreria de embedding.
# Actualmente el modelo predice el texto exacto si se incluyen mas de 4 lineas de cualquier texto, mientras que para busquedas cortas de pocas palabras (1 o un poco mas) el modelo tambien es capaz de encontrar de forma general los textos con mayores coincidencias.

Consulta: carabineros
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
Similitud: 0.9999851584434509
tardes   contraloría general república tomado razón solicitud renuncia general hermes soto isla  desempeñaba general director carabineros  quiero reiterar  forma clara sincera  alto aprecio general soto isla  personal  calidades humanas  profesional  compromiso carabineros  quiero agradecer esfuerzo  trabajo dedicación ejerció cargo general director carabineros  mañana  publicar decreto pone término tramitación proceso  quisiera compartir compatriotas decidido designar cargo general director carabineros chile  general mario rozas córdova  actualmente desempeña director bienestar institución mañana desempeñará general director carabineros chile  general mario rozas córdova ejercido funciones asumido responsabilidades carabineros chile  egresó escuela carabineros año 1987  experiencia operativa territorial distintas destinaciones ancho país  exper